In [1]:
import ee
from datetime import date
import folium
import geemap
import ipyleaflet
import matplotlib.pyplot as plt
from geemap import cartoee
import numpy as np
import cartopy as ccrs
import geemap.colormaps as cm
import pandas as pd
import datetime
from datetime import datetime, timedelta
from collections import OrderedDict
from calendar import monthrange

In [2]:
ee.Initialize()
import sys
sys.path.append('C:\\Rouhin_Lenovo\\US_project\\GEE_SEBAL_Project\\geeSEBAL_copy_edits\\etbrasil\\')
import geesebal
from geesebal import (tools,landsatcollection,masks,meteorology,endmembers, 
evapotranspiration,collection,timeseries,image,ET_Collection_mod)

In [3]:
ls=ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterDate("2022-01-01","2022-01-31").filterMetadata('WRS_PATH', 'equals', 33).filterMetadata('WRS_ROW', 'equals',34).filterMetadata('CLOUD_COVER', 'less_than', 50);
ls.getInfo()
def applyScaleFactors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2);
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0);
    return image.addBands(opticalBands,None, True).addBands(thermalBands,None,True)
ls=ls.map(applyScaleFactors)
ls.first().bandNames().getInfo()
ls.first().select([0,1,2,3,4,5,6,8,17],["UB","B","G","R","NIR","SWIR_1","SWIR_2","ST_B10","pixel_qa"]).bandNames().getInfo()
ls.first().geometry().bounds().getInfo()

{'geodesic': False,
 'type': 'Polygon',
 'coordinates': [[[-106.38388574571538, 36.414947637253185],
   [-103.82729107298924, 36.414947637253185],
   [-103.82729107298924, 38.522523037385156],
   [-106.38388574571538, 38.522523037385156],
   [-106.38388574571538, 36.414947637253185]]]}

In [4]:
ls.toBands().getInfo()

{'type': 'Image',
 'bands': [{'id': 'LC08_033034_20220106_SR_B1',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'dimensions': [7721, 7851],
   'crs': 'EPSG:32613',
   'crs_transform': [30, 0, 374385, 0, -30, 4264815]},
  {'id': 'LC08_033034_20220106_SR_B2',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'dimensions': [7721, 7851],
   'crs': 'EPSG:32613',
   'crs_transform': [30, 0, 374385, 0, -30, 4264815]},
  {'id': 'LC08_033034_20220106_SR_B3',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'dimensions': [7721, 7851],
   'crs': 'EPSG:32613',
   'crs_transform': [30, 0, 374385, 0, -30, 4264815]},
  {'id': 'LC08_033034_20220106_SR_B4',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'dimensions': [7721, 7851],
   'crs': 'EPSG:32613',
   'crs_tran

In [5]:
def split_years(start_date, end_date):
    """
    Function to split query period to bypass computation timed out (300s query limit of gee)
    Takes input as start and end years
    Gives output as list of start and end periods for each year
    """
  
    start, end = [datetime.strptime(_, "%Y-%m-%d") for _ in [start_date, end_date]]
    start = list(OrderedDict(((start + timedelta(_)).strftime(r"%Y-%m-01"), None) for _ in range((end - start).days)).keys())
    end = [item[:-2]+str(monthrange(int(item[:4]),int(item[5:7]))[1]) for item in start] 
#    print(end)
    return start,end
a,b=split_years("2020-01-01","2020-03-31")
# list(a) 
a

['2020-01-01', '2020-02-01', '2020-03-01']

In [6]:
vaira=ee.Geometry.Point([-120.9509216,38.41335394])
ls=ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterDate("2022-01-01","2022-02-01").filterBounds(vaira)
ls.toBands().getInfo()

{'type': 'Image',
 'bands': [{'id': 'LC08_043033_20220112_SR_B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7571, 7721],
   'crs': 'EPSG:32610',
   'crs_transform': [30, 0, 635385, 0, -30, 4425315]},
  {'id': 'LC08_043033_20220112_SR_B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7571, 7721],
   'crs': 'EPSG:32610',
   'crs_transform': [30, 0, 635385, 0, -30, 4425315]},
  {'id': 'LC08_043033_20220112_SR_B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7571, 7721],
   'crs': 'EPSG:32610',
   'crs_transform': [30, 0, 635385, 0, -30, 4425315]},
  {'id': 'LC08_043033_20220112_SR_B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7571, 7721],
   'crs': 'EPSG:32610',
   'crs_transform': [30, 0, 635385, 0, -30, 4425315]

In [7]:
def et_collection_SR(start_date,end_date,lon,lat,scale):
#     try:
#     ls=ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterDate(start_date,end_date).filterMetadata('WRS_PATH', 'equals', path).filterMetadata('WRS_ROW', 'equals',row).filterMetadata('CLOUD_COVER', 'less_than', 10);
    geometry=ee.Geometry.Point([lon,lat])
    ls=ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterDate(start_date,end_date).filterMetadata('CLOUD_COVER', 'less_than',80 )
#     .filterMetadata('CLOUD_COVER', 'less_than', 10);
    ls=ls.filterBounds(geometry)
    def applyScaleFactors(image):
        opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2);
        thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0);
        return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)
    ls=ls.map(applyScaleFactors)
#    ls = ls.filter(ee.Filter.neq('system:index','LC08_038037_20210903'));
    ls_list=ls.aggregate_array('system:index').getInfo()
    print(ls_list)
    count = ls.size().getInfo()
    print("Number of scenes: ", count)
    n=0
    k=0
    lon_cold_pixel=[]
    lat_cold_pixel=[]
    ts_cold_scene=[]
    cold_pixel_lat,cold_pixel_lon,cold_pixel_ndvi,cold_pixel_temp,cold_pixel_sum=[],[],[],[],[]
    hot_pixel_lat,hot_pixel_lon,hot_pixel_ndvi,hot_pixel_temp,hot_pixel_sum=[],[],[],[],[]
    hot_pixel_Rn,hot_pixel_G=[],[]
    zenith_angle=[]
    #====== ITERATIVE PROCESS ======#
    #FOR EACH IMAGE ON THE LIST
    #ESTIMATE ET DAILY IMAGE
    while n < count:
        #GET IMAGE
        image= ls.filterMetadata('system:index','equals',ls_list[n]).first()
        image.getInfo()
        image=ee.Image(image)
            # et=image.Image(image)
        NDVI_cold=5
        Ts_cold=20
        NDVI_hot=10
        Ts_hot=20
        index=image.get('system:index')
        cloud_cover=image.get('CLOUD_COVER')
        LANDSAT_ID=image.get('L1_LANDSAT_PRODUCT_ID').getInfo()
        print(LANDSAT_ID)
        landsat_version=image.get('SATELLITE').getInfo()
        sun_elevation=image.get("SUN_ELEVATION")
        print(sun_elevation.getInfo())
        time_start=image.get('system:time_start')
        date=ee.Date(time_start)
        year=ee.Number(date.get('year'))
        month=ee.Number(date.get('month'))
        day=ee.Number(date.get('day'))
        hour=ee.Number(date.get('hour'))
        minuts = ee.Number(date.get('minutes'))
        print(str(hour.getInfo())+str(minuts.getInfo()))
        crs = image.projection().crs()
        transform=ee.List(ee.Dictionary(ee.Algorithms.Describe(image.projection())).get('transform'))
        date_string=date.format('YYYY-MM-dd').getInfo()
        #ENDMEMBERS
        p_top_NDVI=ee.Number(NDVI_cold)
        p_coldest_Ts=ee.Number(Ts_cold)
        p_lowest_NDVI=ee.Number(NDVI_hot)
        p_hottest_Ts=ee.Number(Ts_hot)
        ls_trial=image.select([0,1,2,3,4,5,6,8,17], ["UB","B","GR","R","NIR","SWIR_1","SWIR_2","ST_B10","pixel_qa"])
#       ls.first_toa=ee.Image('LANDSAT/LC08/C01/T1/'+index.getInfo())
        print(ls_trial.bandNames().getInfo())

#col_rad = ee.Algorithms.Landsat.calibratedRadiance(ls.first_toa)
#col_rad = ls_trial.addBands(col_rad.select([9],["T_RAD"]))
#CLOUD REMOVAL
#ls_trial=ee.ImageCollection(col_rad).map(masks.f_cloudMaskL8_SR)
        ls_trial=masks.f_cloudMaskL8_SR(ls_trial)
#         print("Cloud masking Complete")
        print(ls_trial.bandNames().getInfo())

        #ALBEDO TASUMI ET AL. (2008) METHOD WITH KE ET AL. (2016) COEFFICIENTS
        # ls_trial=ls_trial.map(masks.f_albedoL8)
        ls_trial=masks.f_albedoL8(ls_trial)
        print(ls_trial.bandNames().getInfo())
#         print("Albedo calc done")

        #------ Meteorology
          #GEOMETRY
        geometryReducer=ls_trial.geometry().bounds().getInfo()
#         print("sun elevation check")

        geometry_download=geometryReducer['coordinates']

        # camada_clip=ls_trial.select('BRT').first()
#         camada_clip=ls_trial.select('BRT')
#         sun_elevation=ee.Number(90).subtract(ee.Number(azimuth_angle))
        print(sun_elevation.getInfo())
        #METEOROLOGY PARAMETERS
        col_meteorology= meteorology.get_meteorology(ls_trial,time_start);
        #AIR TEMPERATURE [C]
        T_air = col_meteorology.select('AirT_G');
        print(T_air.bandNames().getInfo())
        #WIND SPEED [M S-1]
        ux= col_meteorology.select('ux_G');
        #RELATIVE HUMIDITY [%]
        UR = col_meteorology.select('RH_G');

        #NET RADIATION 24H [W M-2]
        Rn24hobs = col_meteorology.select('Rn24h_G');

## print("Metorology ready")

        #------
        #------ Elevation
        #SRTM DATA ELEVATION
        SRTM_ELEVATION ='USGS/SRTMGL1_003'
        srtm = ee.Image(SRTM_ELEVATION).clip(geometryReducer);
        z_alt = srtm.select('elevation')
## print(z_alt) 
        ls_trial=tools.fexp_spec_ind(ls_trial)
        ls_trial=tools.LST_DEM_correction(ls_trial, z_alt, T_air, UR,sun_elevation,hour,minuts)
        print("It's a miracle")
## GET IMAGE
## COLD PIXEL
        d_cold_pixel=endmembers.fexp_cold_pixel(ls_trial, geometryReducer, p_top_NDVI, p_coldest_Ts)
        print(d_cold_pixel.getInfo())
## COLD PIXEL NUMBER
        n_Ts_cold = ee.Number(d_cold_pixel.get('temp').getInfo())
##INSTANTANEOUS OUTGOING LONG-WAVE RADIATION [WM-2]
        ls_trial=tools.fexp_radlong_up(ls_trial)
##INSTANTANEOUS INCOMING SHORT-WAVE RADIATION [WM-2]
        ls_trial=tools.fexp_radshort_down(ls_trial,z_alt,T_air,UR, sun_elevation)

## INSTANTANEOUS INCOMING LONGWAVE RADIATION [W M-2]
        ls_trial=tools.fexp_radlong_down(ls_trial, n_Ts_cold)
##INSTANTANEOUS NET RADIATON BALANCE [W M-2]

        ls_trial=tools.fexp_radbalance(ls_trial)

##SOIL HEAT FLUX (G) [W M-2]
        ls_trial=tools.fexp_soil_heat(ls_trial)
##HOT PIXEL
        d_hot_pixel=endmembers.fexp_hot_pixel(ls_trial, geometryReducer,p_lowest_NDVI, p_hottest_Ts)
##SENSIBLE HEAT FLUX (H) [W M-2]
        ls_trial=tools.fexp_sensible_heat_flux(ls_trial, ux, UR,Rn24hobs,n_Ts_cold,
                                       d_hot_pixel, date_string,geometryReducer)

##DAILY EVAPOTRANSPIRATION (ET_24H) [MM DAY-1]
        ls_trial=evapotranspiration.fexp_et(ls_trial,Rn24hobs)
## Store all the values of the cold pixel 
        cold_pixel_lat.append(d_cold_pixel.get("y").getInfo())
        cold_pixel_lon.append(d_cold_pixel.get("x").getInfo())
        cold_pixel_temp.append(d_cold_pixel.get("temp").getInfo())
        cold_pixel_ndvi.append(d_cold_pixel.get("ndvi").getInfo())
        cold_pixel_sum.append(d_cold_pixel.get("sum").getInfo())
## Get info about hot pixl
        hot_pixel_lat.append(d_hot_pixel.get("y").getInfo())
        hot_pixel_lon.append(d_hot_pixel.get("x").getInfo())
        hot_pixel_temp.append(d_hot_pixel.get("temp").getInfo())
        hot_pixel_ndvi.append(d_hot_pixel.get("ndvi").getInfo())
        hot_pixel_sum.append(d_hot_pixel.get("sum").getInfo())
        hot_pixel_Rn.append(d_hot_pixel.get("Rn").getInfo())
        hot_pixel_G.append(d_hot_pixel.get("G").getInfo())
        zenith_angle.append(90-sun_elevation.getInfo())

        NAME_FINAL=LANDSAT_ID[:5]+LANDSAT_ID[10:17]+LANDSAT_ID[17:25]
        if k ==0:



              #AIR TEMPERATURE [C]
#             Rl_out=ls_trial.select(['Rl_up'],[NAME_FINAL])
#             Rl_in=ls_trial.select(['Rn'],[NAME_FINAL])
#             tao_sw_1=ls_trial.select(["T_LST_DEM"],[NAME_FINAL])
#             Rn=ls_trial.select(['Rn'],[NAME_FINAL])
#             G=ls_trial.select(['G'],[NAME_FINAL])
#             H=ls_trial.select(['H'],[NAME_FINAL])
#             LE_inst=ls_trial.select(['LE'],[NAME_FINAL])
#             Rn_daily=ls_trial.select(['ST_B10'],[NAME_FINAL])
#             ET=ls_trial.select(['ET_24h',"ST_B10","H","G"],[NAME_FINAL])
            new_ls=ee.List([])
            met=ee.List([])
            new_ls=new_ls.add(ls_trial.select("ET_24h"))
            met=met.add(col_meteorology.select("Rn24h_G","AirT_G","RH_G","ux_G","SW_Down"))
#             collectionFromConstructor=ee.ImageCollection()
#             collectionFromConstructor = ee.ImageCollection(ls_trial)



        else:
#             Rl_out=Rl_out.addBands(ls_trial.select(['Rl_up'],[NAME_FINAL]))
#             Rl_in=Rl_in.addBands(ls_trial.select(['Rn'],[NAME_FINAL]))
#             tao_sw_1=tao_sw_1.addBands(ls_trial.select(["T_LST_DEM"],[NAME_FINAL]))
#             Rn=Rn.addBands(ls_trial.select(['Rn'],[NAME_FINAL]))
#             G=G.addBands(ls_trial.select(['G'],[NAME_FINAL]))
#             H=H.addBands(ls_trial.select(['H'],[NAME_FINAL]))
#             LE_inst=LE_inst.addBands(ls_trial.select(['LE'],[NAME_FINAL]))
#             Rn_daily=Rn_daily.addBands(ls_trial.select(['ST_B10'],[NAME_FINAL]))
#             ET=ET.addBands(ls_trial.select(['ET_24h',"ST_B10","H","G"],[NAME_FINAL]))
#             collectionFromConstructor = collectionFromConstructor.merge(ee.ImageCollection(ls_trial.select(['ET_24h',"ST_B10","H","G"],[NAME_FINAL])))
#             print('collectionFromConstructor: ', collectionFromConstructor);
            new_ls=new_ls.add(ls_trial.select("ET_24h"))
            met=met.add(col_meteorology.select("Rn24h_G","AirT_G","RH_G","ux_G","SW_Down"))





        k=k+1
        print(k)
        n=n+1
        print(n)
        ## Convert to dataframe 
        et_collection=ee.ImageCollection(new_ls)
        met_collection=ee.ImageCollection(met)

        ##Stuff t

        region = et_collection.getRegion(geometry, int(scale)).getInfo()
        era5_met=met_collection.getRegion(geometry, int(scale)).getInfo()
#     except:
#         return pd.DataFrame()
#       print('Value = %0.2f'%region[1][4])
# stuff the values in a dataframe for convenience      
    df = pd.DataFrame.from_records(region[1:len(region)])
    df_met = pd.DataFrame.from_records(era5_met[1:len(era5_met)])

    if df.shape == (0,0):
        return pd.DataFrame()
    else:

        # use the first list item as column names
        df.columns = region[0]
        df_met.columns=era5_met[0]
        df_met=df_met.drop(["time"],axis=1)
        print(df_met)
        df=pd.concat([df,df_met],axis=1)
#         cold_pixel_lat,cold_pixel_lon,cold_pixel_ndvi,cold_pixel_temp,cold_pixel_sum
        df["cold_pixel_lat"]=cold_pixel_lat
        df["cold_pixel_lon"]=cold_pixel_lon
        df["cold_pixel_ndvi"]=cold_pixel_ndvi
        df["cold_pixel_sum"]=cold_pixel_sum
        df["cold_pixel_temp"]=cold_pixel_temp

## Hot pixel
        df["hot_pixel_sum"]=hot_pixel_sum
        df["hot_pixel_lat"]=hot_pixel_lat
        df["hot_pixel_lon"]=hot_pixel_lon
        df["hot_pixel_ndvi"]=hot_pixel_ndvi
        df["hot_pixel_Rn"]=hot_pixel_Rn
        df["hot_pixel_G"]=hot_pixel_G
        df["hot_pixel_temp"]=hot_pixel_temp
      #drop id column(little value / overhead)
      #df.drop('id',axis=1,inplace=True)

      # divide the time field by 1000 as in milliseconds
      # while datetime takes seconds to convert unix time
      # to dates
        df.time = df.time / 1000
        df['time'] = pd.to_datetime(df['time'], unit = 's')
        df.rename(columns = {'time': 'date'}, inplace = True)
        df.sort_values(by = 'date')
        
      # add the product name and latitude, longitude as a column
      # just to make sense of the returned data after the fact
#      df['product'] = pd.Series(product, index = df.index)

      # return output
        return df
        
#     return(albedo,tao_sw)
#     return(new_ls)
# re
# et=et_collection_SR("2021-01-01","2021-03-20",-121.4993,38.0992,30)
# rs=et_collection_SR("2021-01-01","2021-12-31",44,33)
# alb_gee=alb_gee.reproject(crs="EPSG:4326",scale=30)
# tao_gee=tao_gee.reproject(crs="EPSG:4326",scale=30)
# srtm=srtm.reproject(crs="EPSG:4326",scale=30)


In [8]:
def call_et_func(lon,lat,start_date,end_date,scale):
        start,end=split_years(start_date,end_date)
        print(len(start))
        concat=pd.DataFrame()
        for i in range(len(start)):
                try:
                    df_sub=et_collection_SR(start[i],end[i],lon,lat,scale)
                    concat = pd.concat([concat, df_sub], axis = 0) 
                except NameError:
                    print("Error: check input parameters")
        return concat
var=call_et_func(-120.9509216,38.41335394,"2015-01-01","2015-01-03",30)

1


KeyboardInterrupt: 

In [19]:
var

,id,longitude,latitude,date,ET_24h,id,longitude,latitude,Rn24h_G,AirT_G,...,cold_pixel_ndvi,cold_pixel_sum,cold_pixel_temp,hot_pixel_sum,hot_pixel_lat,hot_pixel_lon,hot_pixel_ndvi,hot_pixel_Rn,hot_pixel_G,hot_pixel_temp
0,LC08_043033_20150125,-120.950922,38.413354,2015-01-25 18:39:27.064000000,1.046295,0,-120.950922,38.413354,44.802651,14.882873,...,0.904567,403872,277.630347,595612,39.575838,-119.012860,0.072029,266.770516,35.140664,292.980674
1,LC08_043034_20150125,-120.950922,38.413354,2015-01-25 18:39:50.951000064,NaN,1,-120.950922,38.413354,NaN,14.886759,...,0.966130,190156,286.342880,314876,37.947745,-119.948667,0.389565,257.666888,53.379813,303.969609


In [12]:
def call_et_func(lon,lat,start_date,end_date,scale):
        start,end=split_years(start_date,end_date)
        print(len(start))
        concat=pd.DataFrame()
        for i in range(len(start)):
                try:
                    df_sub=et_collection_SR(start[i],end[i],lon,lat,scale)
                    concat = pd.concat([concat, df_sub], axis = 0) 
                except NameError:
                    print("Error: check input parameters")
        return concat
# rice1=call_et_func(30.17486,-2.04951, "2022-08-01","2023-03-28",30)
# alfa_south=call_et_func(-114.739405,33.438965,"2021-01-01","2021-12-31",30)
# coffee=call_et_func( 29.63955 ,-2.46906,"2022-08-01","2023-03-31",30)
alfa_north=call_et_func(-114.715314,33.547044, "2021-01-01","2021-12-31",30)

# 33.547044, -114.715314 alfa north
# -2.46906, 29.63955 coffee
# srg 2023-4-13 33.438965, -114.739405

12
['LC08_038037_20210106', 'LC08_038037_20210122', 'LC08_039037_20210113', 'LC08_039037_20210129']
Number of scenes:  4
LC08_L1TP_038037_20210106_20210307_02_T1
30.25946943
1810
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
30.25946943
['AirT_G']
It's a miracle
{'ndvi': 0.4821678698062897, 'sum': 163858, 'temp': 286.82823848, 'x': -114.57844373483498, 'y': 33.674792148331846}
1
1
LC08_L1TP_038037_20210122_20210305_02_T1
32.40087714
1810
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
32.40087714
['AirT_G']
It's a miracle
{'ndvi': 0.6655802130699158, 'sum': 187932, 'temp': 290.6085686, 'x': -115.10700906745154, 'y': 32.52527162292864}
2
2

In [14]:
# rice1.to_csv("C:\\Users\\rouhi\\Downloads\\Rice.csv")
# alfa_north.to_csv("C:\\Users\\rouhi\\Downloads\\alfa.csv")
# coffee.to_csv("C:\\Users\\rouhi\\Downloads\\coffee.csv")
alfa_north.to_csv("C:\\Users\\rouhi\\Downloads\\alfa_north.csv")


In [ ]:
var.to_csv("C:\\Rouhin_Lenovo\\US_project\\GEE_SEBAL_Project\\Csv_Files\\SEBAL_Results_CONUS\\US-Var_met.csv")

In [ ]:
var[var["UB"].isna()==True]

In [ ]:
srg.to_csv("C:\\Rouhin_Lenovo\\US_project\\GEE_SEBAL_Project\\Csv_Files\\SEBAL_Results_CONUS\\US-SRG_met.csv")

In [ ]:
ee.ImageCollection(et).toBands().reduceRegion(reducer= ee.Reducer.first(),
    geometry=ee.Geometry.Point([-121.4993,38.0992]),
    scale=int(30),
    crs='EPSG:4326',
    maxPixels=9000000000).getInfo()

## Get until net radiation 

In [7]:
def et_collection_SR(start_date,end_date,lon,lat,scale):
#     try:
#     ls=ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterDate(start_date,end_date).filterMetadata('WRS_PATH', 'equals', path).filterMetadata('WRS_ROW', 'equals',row).filterMetadata('CLOUD_COVER', 'less_than', 10);
    geometry=ee.Geometry.Point([lon,lat])
    ls=ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterDate(start_date,end_date).filterMetadata('CLOUD_COVER', 'less_than',80 )
#     .filterMetadata('CLOUD_COVER', 'less_than', 10);
    ls=ls.filterBounds(geometry)
    def applyScaleFactors(image):
        opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2);
        thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0);
        return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)
    ls=ls.map(applyScaleFactors)
#    ls = ls.filter(ee.Filter.neq('system:index','LC08_038037_20210903'));
    ls_list=ls.aggregate_array('system:index').getInfo()
    print(ls_list)
    count = ls.size().getInfo()
    print("Number of scenes: ", count)
    n=0
    k=0
    lon_cold_pixel=[]
    lat_cold_pixel=[]
    ts_cold_scene=[]
    cold_pixel_lat,cold_pixel_lon,cold_pixel_ndvi,cold_pixel_temp,cold_pixel_sum=[],[],[],[],[]
    hot_pixel_lat,hot_pixel_lon,hot_pixel_ndvi,hot_pixel_temp,hot_pixel_sum=[],[],[],[],[]
    hot_pixel_Rn,hot_pixel_G=[],[]
    zenith_angle=[]
    #====== ITERATIVE PROCESS ======#
    #FOR EACH IMAGE ON THE LIST
    #ESTIMATE ET DAILY IMAGE
    while n < count:
        #GET IMAGE
        image= ls.filterMetadata('system:index','equals',ls_list[n]).first()
        image.getInfo()
        image=ee.Image(image)
            # et=image.Image(image)
        NDVI_cold=5
        Ts_cold=20
        NDVI_hot=10
        Ts_hot=20
        index=image.get('system:index')
        cloud_cover=image.get('CLOUD_COVER')
        LANDSAT_ID=image.get('L1_LANDSAT_PRODUCT_ID').getInfo()
        print(LANDSAT_ID)
        landsat_version=image.get('SATELLITE').getInfo()
        sun_elevation=image.get("SUN_ELEVATION")
        print(sun_elevation.getInfo())
        time_start=image.get('system:time_start')
        date=ee.Date(time_start)
        year=ee.Number(date.get('year'))
        month=ee.Number(date.get('month'))
        day=ee.Number(date.get('day'))
        hour=ee.Number(date.get('hour'))
        minuts = ee.Number(date.get('minutes'))
        print(str(hour.getInfo())+str(minuts.getInfo()))
        crs = image.projection().crs()
        transform=ee.List(ee.Dictionary(ee.Algorithms.Describe(image.projection())).get('transform'))
        date_string=date.format('YYYY-MM-dd').getInfo()
        #ENDMEMBERS
        p_top_NDVI=ee.Number(NDVI_cold)
        p_coldest_Ts=ee.Number(Ts_cold)
        p_lowest_NDVI=ee.Number(NDVI_hot)
        p_hottest_Ts=ee.Number(Ts_hot)
        ls_trial=image.select([0,1,2,3,4,5,6,8,17], ["UB","B","GR","R","NIR","SWIR_1","SWIR_2","ST_B10","pixel_qa"])
#       ls.first_toa=ee.Image('LANDSAT/LC08/C01/T1/'+index.getInfo())
        print(ls_trial.bandNames().getInfo())

#col_rad = ee.Algorithms.Landsat.calibratedRadiance(ls.first_toa)
#col_rad = ls_trial.addBands(col_rad.select([9],["T_RAD"]))
#CLOUD REMOVAL
#ls_trial=ee.ImageCollection(col_rad).map(masks.f_cloudMaskL8_SR)
        ls_trial=masks.f_cloudMaskL8_SR(ls_trial)
#         print("Cloud masking Complete")
        print(ls_trial.bandNames().getInfo())

        #ALBEDO TASUMI ET AL. (2008) METHOD WITH KE ET AL. (2016) COEFFICIENTS
        # ls_trial=ls_trial.map(masks.f_albedoL8)
        ls_trial=masks.f_albedoL8(ls_trial)
        print(ls_trial.bandNames().getInfo())
#         print("Albedo calc done")

        #------ Meteorology
          #GEOMETRY
        geometryReducer=ls_trial.geometry().bounds().getInfo()
#         print("sun elevation check")

        geometry_download=geometryReducer['coordinates']

        # camada_clip=ls_trial.select('BRT').first()
#         camada_clip=ls_trial.select('BRT')
#         sun_elevation=ee.Number(90).subtract(ee.Number(azimuth_angle))
        print(sun_elevation.getInfo())
        #METEOROLOGY PARAMETERS
        col_meteorology= meteorology.get_meteorology(ls_trial,time_start);
        #AIR TEMPERATURE [C]
        T_air = col_meteorology.select('AirT_G');
        print(T_air.bandNames().getInfo())
        #WIND SPEED [M S-1]
        ux= col_meteorology.select('ux_G');
        #RELATIVE HUMIDITY [%]
        UR = col_meteorology.select('RH_G');

        #NET RADIATION 24H [W M-2]
        Rn24hobs = col_meteorology.select('Rn24h_G');
## print("Metorology ready")

        #------
        #------ Elevation
        #SRTM DATA ELEVATION
        SRTM_ELEVATION ='USGS/SRTMGL1_003'
        srtm = ee.Image(SRTM_ELEVATION).clip(geometryReducer);
        z_alt = srtm.select('elevation')
## print(z_alt) 
        ls_trial=tools.fexp_spec_ind(ls_trial)
        ls_trial=tools.LST_DEM_correction(ls_trial, z_alt, T_air, UR,sun_elevation,hour,minuts)
        print("It's a miracle")
## GET IMAGE
## COLD PIXEL
        d_cold_pixel=endmembers.fexp_cold_pixel(ls_trial, geometryReducer, p_top_NDVI, p_coldest_Ts)
        print(d_cold_pixel.getInfo())
## COLD PIXEL NUMBER
        n_Ts_cold = ee.Number(d_cold_pixel.get('temp').getInfo())
##INSTANTANEOUS OUTGOING LONG-WAVE RADIATION [WM-2]
        ls_trial=tools.fexp_radlong_up(ls_trial)
##INSTANTANEOUS INCOMING SHORT-WAVE RADIATION [WM-2]
        ls_trial=tools.fexp_radshort_down(ls_trial,z_alt,T_air,UR, sun_elevation)

## INSTANTANEOUS INCOMING LONGWAVE RADIATION [W M-2]
        ls_trial=tools.fexp_radlong_down(ls_trial, n_Ts_cold)
##INSTANTANEOUS NET RADIATON BALANCE [W M-2]

        ls_trial=tools.fexp_radbalance(ls_trial)

##SOIL HEAT FLUX (G) [W M-2]
        ls_trial=tools.fexp_soil_heat(ls_trial)
##HOT PIXEL
#         d_hot_pixel=endmembers.fexp_hot_pixel(ls_trial, geometryReducer,p_lowest_NDVI, p_hottest_Ts)
# ##SENSIBLE HEAT FLUX (H) [W M-2]
#         ls_trial=tools.fexp_sensible_heat_flux(ls_trial, ux, UR,Rn24hobs,n_Ts_cold,
#                                        d_hot_pixel, date_string,geometryReducer)

##DAILY EVAPOTRANSPIRATION (ET_24H) [MM DAY-1]
#         ls_trial=evapotranspiration.fexp_et(ls_trial,Rn24hobs)
## Store all the values of the cold pixel 
        cold_pixel_lat.append(d_cold_pixel.get("y").getInfo())
        cold_pixel_lon.append(d_cold_pixel.get("x").getInfo())
        cold_pixel_temp.append(d_cold_pixel.get("temp").getInfo())
        cold_pixel_ndvi.append(d_cold_pixel.get("ndvi").getInfo())
        cold_pixel_sum.append(d_cold_pixel.get("sum").getInfo())
## Get info about hot pixl
#         hot_pixel_lat.append(d_hot_pixel.get("y").getInfo())
#         hot_pixel_lon.append(d_hot_pixel.get("x").getInfo())
#         hot_pixel_temp.append(d_hot_pixel.get("temp").getInfo())
#         hot_pixel_ndvi.append(d_hot_pixel.get("ndvi").getInfo())
#         hot_pixel_sum.append(d_hot_pixel.get("sum").getInfo())
#         hot_pixel_Rn.append(d_hot_pixel.get("Rn").getInfo())
#         hot_pixel_G.append(d_hot_pixel.get("G").getInfo())
        zenith_angle.append(90-sun_elevation.getInfo())

        NAME_FINAL=LANDSAT_ID[:5]+LANDSAT_ID[10:17]+LANDSAT_ID[17:25]
        if k ==0:



              #AIR TEMPERATURE [C]
#             Rl_out=ls_trial.select(['Rl_up'],[NAME_FINAL])
#             Rl_in=ls_trial.select(['Rn'],[NAME_FINAL])
#             tao_sw_1=ls_trial.select(["T_LST_DEM"],[NAME_FINAL])
#             Rn=ls_trial.select(['Rn'],[NAME_FINAL])
#             G=ls_trial.select(['G'],[NAME_FINAL])
#             H=ls_trial.select(['H'],[NAME_FINAL])
#             LE_inst=ls_trial.select(['LE'],[NAME_FINAL])
#             Rn_daily=ls_trial.select(['ST_B10'],[NAME_FINAL])
#             ET=ls_trial.select(['ET_24h',"ST_B10","H","G"],[NAME_FINAL])
            new_ls=ee.List([])
            met=ee.List([])
            new_ls=new_ls.add(ls_trial)
            met=met.add(col_meteorology.select("Rn24h_G","AirT_G","RH_G","ux_G","SW_Down"))
#             collectionFromConstructor=ee.ImageCollection()
#             collectionFromConstructor = ee.ImageCollection(ls_trial)



        else:
#             Rl_out=Rl_out.addBands(ls_trial.select(['Rl_up'],[NAME_FINAL]))
#             Rl_in=Rl_in.addBands(ls_trial.select(['Rn'],[NAME_FINAL]))
#             tao_sw_1=tao_sw_1.addBands(ls_trial.select(["T_LST_DEM"],[NAME_FINAL]))
#             Rn=Rn.addBands(ls_trial.select(['Rn'],[NAME_FINAL]))
#             G=G.addBands(ls_trial.select(['G'],[NAME_FINAL]))
#             H=H.addBands(ls_trial.select(['H'],[NAME_FINAL]))
#             LE_inst=LE_inst.addBands(ls_trial.select(['LE'],[NAME_FINAL]))
#             Rn_daily=Rn_daily.addBands(ls_trial.select(['ST_B10'],[NAME_FINAL]))
#             ET=ET.addBands(ls_trial.select(['ET_24h',"ST_B10","H","G"],[NAME_FINAL]))
#             collectionFromConstructor = collectionFromConstructor.merge(ee.ImageCollection(ls_trial.select(['ET_24h',"ST_B10","H","G"],[NAME_FINAL])))
#             print('collectionFromConstructor: ', collectionFromConstructor);
            new_ls=new_ls.add(ls_trial)
            met=met.add(col_meteorology.select("Rn24h_G","AirT_G","RH_G","ux_G","SW_Down"))





        k=k+1
        print(k)
        n=n+1
        print(n)
        ## Convert to dataframe 
        et_collection=ee.ImageCollection(new_ls)
        met_collection=ee.ImageCollection(met)

        ##Stuff t

        region = et_collection.getRegion(geometry, int(scale)).getInfo()
        era5_met=met_collection.getRegion(geometry, int(scale)).getInfo()
#     except:
#         return pd.DataFrame()
#       print('Value = %0.2f'%region[1][4])
# stuff the values in a dataframe for convenience      
    df = pd.DataFrame.from_records(region[1:len(region)])
    df_met = pd.DataFrame.from_records(era5_met[1:len(era5_met)])

    if df.shape == (0,0):
        return pd.DataFrame()
    else:

        # use the first list item as column names
        df.columns = region[0]
        df_met.columns=era5_met[0]
        df_met=df_met.drop(["time"],axis=1)
        print(df_met)
        df=pd.concat([df,df_met],axis=1)
#         cold_pixel_lat,cold_pixel_lon,cold_pixel_ndvi,cold_pixel_temp,cold_pixel_sum
        df["cold_pixel_lat"]=cold_pixel_lat
        df["cold_pixel_lon"]=cold_pixel_lon
        df["cold_pixel_ndvi"]=cold_pixel_ndvi
        df["cold_pixel_sum"]=cold_pixel_sum
        df["cold_pixel_temp"]=cold_pixel_temp
        df["zenith_angle"]=zenith_angle
## Hot pixel
#         df["hot_pixel_sum"]=hot_pixel_sum
#         df["hot_pixel_lat"]=hot_pixel_lat
#         df["hot_pixel_lon"]=hot_pixel_lon
#         df["hot_pixel_ndvi"]=hot_pixel_ndvi
#         df["hot_pixel_Rn"]=hot_pixel_Rn
#         df["hot_pixel_G"]=hot_pixel_G
#         df["hot_pixel_temp"]=hot_pixel_temp

      # drop id column (little value / overhead)
#       df.drop('id', axis=1, inplace=True)

      # divide the time field by 1000 as in milliseconds
      # while datetime takes seconds to convert unix time
      # to dates
        df.time = df.time / 1000
        df['time'] = pd.to_datetime(df['time'], unit = 's')
        df.rename(columns = {'time': 'date'}, inplace = True)
        df.sort_values(by = 'date')
        
      # add the product name and latitude, longitude as a column
      # just to make sense of the returned data after the fact
#      df['product'] = pd.Series(product, index = df.index)

      # return output
        return df
        
#     return(albedo,tao_sw)
#     return(new_ls)
# re
# et=et_collection_SR("2021-01-01","2021-03-20",-121.4993,38.0992,30)

# rs=et_collection_SR("2021-01-01","2021-12-31",44,33)
# alb_gee=alb_gee.reproject(crs="EPSG:4326",scale=30)
# tao_gee=tao_gee.reproject(crs="EPSG:4326",scale=30)
# srtm=srtm.reproject(crs="EPSG:4326",scale=30)


In [8]:
## Read ameriflux 
st_am=pd.read_csv("C:\\Rouhin_Lenovo\\US_project\\Untitled Folder\\Station_List\\AMERIFLUX_SWC_Met_US.csv",encoding="unicode_escape")

In [9]:
st_am

,Name,Data Use Policy,AmeriFlux BASE Data,Lat,Long,Elev (m),Veg,Clim,MAT (°C),MAP (mm),AmeriFlux BASE Start,AmeriFlux BASE End
0,US-ADR,Amargosa Desert Research Site (ADRS),Legacy,36.7653,-116.6933,842.0,BSV,Bwh,17.50,110.00,2011,2017
1,US-ARM,ARM Southern Great Plains site- Lamont,CC-BY-4.0,36.6058,-97.4888,314.0,CRO,Cfa,14.76,843.00,2003,2021
2,US-Bi1,Bouldin Island Alfalfa,CC-BY-4.0,38.0992,-121.4993,-2.7,CRO,Csa,16.00,338.00,2016,2022
3,US-Bi2,Bouldin Island corn,CC-BY-4.0,38.1091,-121.5351,-5.0,CRO,Csa,16.00,338.00,2017,2022
4,US-CMW,Charleston Mesquite Woodland,CC-BY-4.0,31.6637,-110.1777,1199.0,DBF,Bsh,17.00,288.00,2000,2021
...,...,...,...,...,...,...,...,...,...,...,...,...
90,US-xUK,NEON The University of Kansas Field Station (U...,CC-BY-4.0,39.0404,-95.1921,335.0,DBF,Cfa,12.70,990.63,2017,2022
91,US-xUN,NEON University of Notre Dame Environmental Re...,CC-BY-4.0,46.2339,-89.5373,518.0,MF,Dfb,4.30,802.30,2017,2022
92,US-xWD,NEON Woodworth (WOOD),CC-BY-4.0,47.1282,-99.2414,579.0,GRA,Dfb,4.90,493.76,2017,2022
93,US-xWR,NEON Wind River Experimental Forest (WREF),CC-BY-4.0,45.8205,-121.9519,407.0,ENF,Csb,9.20,2225.01,2018,2022


In [10]:
 def call_et_func(lon,lat,start_date,end_date,scale):
        start,end=split_years(start_date,end_date)
        print(len(start))
        concat=pd.DataFrame()
        for i in range(len(start)):
                try:
                    df_sub=et_collection_SR(start[i],end[i],lon,lat,scale)
                    concat = pd.concat([concat, df_sub], axis = 0) 
                except NameError:
                    print("Error: check input parameters")
        return concat
# srg=call_et_func(-110.8277,31.7894,"2015-01-01","2015-01-31",30)
# srg

In [ ]:
srg.to_csv("C:\\Rouhin_Lenovo\\US_project\\GEE_SEBAL_Project\\Csv_Files\\Netrad_AM\\"+st_am["Name"].iloc[0]+".csv")

In [ ]:
for i in reversed(range(st_am.shape[0]-33)):
    print(st_am["Long"].iloc[i],st_am["Lat"].iloc[i])
    try:
        print(st_am['Name'].iloc[i])
        tmp=call_et_func(st_am["Long"].iloc[i],st_am["Lat"].iloc[i],"2015-01-01","2022-12-31",30)
        tmp.to_csv("C:\\Rouhin_Lenovo\\US_project\\GEE_SEBAL_Project\\Csv_Files\\Netrad_AM\\"+st_am["Name"].iloc[i]+".csv")
    except NameError:
        print("Error: check input parameters")   

-104.7456 40.8155
US-xCP
96
['LC08_033032_20150119', 'LC08_034031_20150110', 'LC08_034031_20150126', 'LC08_034032_20150110', 'LC08_034032_20150126']
Number of scenes:  5
LC08_L1TP_033032_20150119_20200910_02_T1
25.82782393
1737
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
25.82782393
['AirT_G']
It's a miracle
{'ndvi': 0.8472135066986084, 'sum': 347874, 'temp': 275.44281386, 'x': -105.4035940410984, 'y': 39.68198977316676}
1
1
LC08_L1TP_034031_20150110_20200910_02_T1
23.29881162
1743
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
23.29881162
['AirT_G']
It's a miracle
{'ndvi': 0.595510721206665, 'sum': 57224, 'temp': 268.23079166, 'x': -1

Number of scenes:  5
LC08_L1TP_033032_20150511_20200909_02_T1
62.10764305
1736
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
62.10764305
['AirT_G']
It's a miracle
{'ndvi': 0.8727061152458191, 'sum': 365866, 'temp': 287.6861615, 'x': -103.13182425340842, 'y': 40.8357189456842}
1
1
LC08_L1TP_033032_20150527_20200909_02_T1
64.83718959
1736
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
64.83718959
['AirT_G']
It's a miracle
{'ndvi': 0.8704758882522583, 'sum': 332561, 'temp': 293.59250006, 'x': -102.96467733149159, 'y': 40.879639757063224}
2
2
LC08_L1TP_034031_20150502_20200909_02_T1
58.97289758
1742
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'S

Number of scenes:  5
LC08_L1TP_033032_20150815_20200908_02_T1
57.84259003
1737
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
57.84259003
['AirT_G']
It's a miracle
{'ndvi': 0.9134200811386108, 'sum': 414734, 'temp': 303.09459566, 'x': -104.77207953443938, 'y': 40.54387661009036}
1
1
LC08_L1TP_034031_20150806_20200908_02_T1
59.04294933
1742
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
59.04294933
['AirT_G']
It's a miracle
{'ndvi': 0.6676519513130188, 'sum': 493067, 'temp': 290.46159374, 'x': -104.23625289995714, 'y': 42.12295368668126}
2
2
LC08_L1TP_034031_20150822_20200908_02_T1
55.09912003
1742
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', '

Number of scenes:  6
LC08_L1TP_033032_20151103_20200908_02_T1
32.64329819
1737
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
32.64329819
['AirT_G']
It's a miracle
{'ndvi': 0.6648774743080139, 'sum': 526717, 'temp': 284.8833851, 'x': -105.51125327314672, 'y': 39.7172461200329}
1
1
LC08_L1TP_033032_20151119_20200908_02_T1
28.28886507
1737
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
28.28886507
['AirT_G']
It's a miracle
{'ndvi': 0.8660128116607666, 'sum': 241303, 'temp': 273.17324858, 'x': -105.32752266622911, 'y': 39.64871093177385}
2
2
LC08_L1TP_034031_20151110_20200908_02_T1
29.33142754
1743
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SW

Number of scenes:  3
LC08_L1TP_033032_20160207_20200907_02_T1
30.23458539
1737
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
30.23458539
['AirT_G']
It's a miracle
{'ndvi': 0.967676043510437, 'sum': 1001, 'temp': 268.77767486, 'x': -105.33714603365128, 'y': 40.32928311805198}
1
1
LC08_L1TP_033032_20160223_20200907_02_T1
35.39610044
1737
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
35.39610044
['AirT_G']
It's a miracle
{'ndvi': 0.5240710973739624, 'sum': 412395, 'temp': 276.7143173, 'x': -103.96551675058318, 'y': 39.81647581410094}
2
2
LC08_L1TP_034031_20160214_20200907_02_T1
31.21532958
1743
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR

...

LC08_L1TP_044033_20170427_20200904_02_T1
59.64273563
1844
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
59.64273563
['AirT_G']
It's a miracle
{'ndvi': 0.8679869771003723, 'sum': 357739, 'temp': 292.85762576, 'x': -122.51750804342056, 'y': 38.607118997449035}
1
1
LC08_L1TP_044034_20170427_20200904_02_T1
60.53619015
1845
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
60.53619015
['AirT_G']
It's a miracle
{'ndvi': 0.8821284174919128, 'sum': 365825, 'temp': 289.52847427999995, 'x': -122.29562054579891, 'y': 37.24703717215767}
2
2
  id   longitude   latitude     Rn24h_G     AirT_G       RH_G      ux_G  \
0  0 -121.647026  38.115293  177.68978

Number of scenes:  4
LC08_L1TP_044033_20170902_20200903_02_T1
53.87791981
1845
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
53.87791981
['AirT_G']
It's a miracle
{'ndvi': 0.8353360295295715, 'sum': 527974, 'temp': 304.89247418, 'x': -121.059322109499, 'y': 39.68512266135574}
1
1
LC08_L1TP_044033_20170918_20200903_02_T1
48.94680465
1845
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
48.94680465
['AirT_G']
It's a miracle
{'ndvi': 0.8399806022644043, 'sum': 306778, 'temp': 294.21116168000003, 'x': -122.63395124646678, 'y': 39.14958323763962}
2
2
LC08_L1TP_044034_20170902_20200903_02_T1
54.87760466
1846
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1

27.37906261
['AirT_G']
It's a miracle
{'ndvi': 0.9011895060539246, 'sum': 309509, 'temp': 284.07331436, 'x': -122.09984066827732, 'y': 37.098604396758496}
3
3
LC08_L1TP_044034_20171223_20200902_02_T1
26.15337922
1846
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
26.15337922
['AirT_G']
It's a miracle
{'ndvi': 0.7937754392623901, 'sum': 27833, 'temp': 279.33935666, 'x': -122.10401813831706, 'y': 37.030489991909114}
4
4
  id   longitude   latitude    Rn24h_G     AirT_G       RH_G      ux_G  \
0  0 -121.647026  38.115293  32.977293  11.703014  51.038435  1.111101   
1  1 -121.647026  38.115293        NaN   9.100217  62.385022  0.604838   
2  2 -121.647026  38.115293  32.894317  11.713968  51.001396  1.111505   
3  3 -121.647026  38.115293        NaN   9.107867  62.336606  0.604862   

      SW_Down  
0  124.085296 

['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
66.69809212
['AirT_G']
It's a miracle
{'ndvi': 0.8378313183784485, 'sum': 428742, 'temp': 293.6335163, 'x': -122.67352397704896, 'y': 37.932535639821}
3
3
LC08_L1TP_044034_20180617_20200831_02_T1
67.09744516
1844
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
67.09744516
['AirT_G']
It's a miracle
{'ndvi': 0.8441581130027771, 'sum': 191002, 'temp': 296.1970313, 'x': -122.19815498879049, 'y': 37.94810186366679}
4
4
  id   longitude   latitude     Rn24h_G     AirT_G       RH_G      ux_G  \
0  0 -121.647026  38.115293  197.881535  24.605120  32.670653  1.658721   
1  1 -121.647026  38.115293         NaN  21.523105  42.528577  0.761656   
2  2 -121.647026  38.115293  197.897850  24.616748  32.638132  1.659467   
3  3 -121.647026  38.11529

Number of scenes:  4
LC08_L1TP_044033_20181007_20200830_02_T1
42.62885674
1845
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
42.62885674
['AirT_G']
It's a miracle
{'ndvi': 0.8958414793014526, 'sum': 524311, 'temp': 286.24375706, 'x': -120.96360315864058, 'y': 39.58537167280734}
1
1
LC08_L1TP_044033_20181023_20200830_02_T1
37.25939087
1845
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
37.25939087
['AirT_G']
It's a miracle
{'ndvi': 0.7974075675010681, 'sum': 56057, 'temp': 285.84043069999996, 'x': -120.89429000691364, 'y': 39.080046824119414}
2
2
LC08_L1TP_044034_20181007_20200830_02_T1
43.82626863
1845
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR

In [38]:
st_am.shape[0]-21
print(st_am['Name'].iloc[st_am.shape[0]-33])


US-xDC
